# Importing Packages

In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(15)
import json
import shutil
from termcolor import colored
from scipy.stats import skew
import warnings
warnings.filterwarnings('ignore')

In [25]:
def print_heading(text):
  print(colored(text, attrs=['bold','blink'], color='yellow'))

def dash_line():
    print("-----------------------------------------------------------------")

# Importing Data

In [26]:
data = pd.read_csv("/content/drive/MyDrive/Datasets/Final_Data.csv")
orignal_data = pd.read_csv("/content/drive/MyDrive/Datasets/Final_Data_with_Features.csv")
data.drop(columns='Unnamed: 0',inplace=True)
orignal_data.drop(columns='Unnamed: 0',inplace=True)
data.dropna(subset=['pid'],inplace=True,axis=0)
data.sort_values(by='pid',inplace=True)
data.fillna(0,inplace=True)

In [27]:
data.head()

,pid,track_uri,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,time_signature,subjectivity_high,subjectivity_low,subjectivity_medium,polarity_Negative,polarity_Neutral,polarity_Positive
0,0.0,0UaMYEvWZi0ZqiDOoHU3YI,0.720430,0.912210,0.813,0.363636,0.842733,0.0,0.595529,0.031225,...,0.465860,0.810,0.540594,0.8,0.0,1.0,0.0,0.0,1.0,0.0
1189,0.0,4E5P1XyAFtrjpiIxkydly4,0.806452,0.712412,0.751,0.818182,0.855192,1.0,0.520852,0.173695,...,0.640217,0.195,0.392240,0.8,0.0,1.0,0.0,0.0,1.0,0.0
1243,0.0,3H1LCvO3fVsK2HPguhbml0,0.000000,0.675076,0.822,1.000000,0.887614,1.0,0.459805,0.034036,...,0.693271,0.430,0.517110,0.8,0.0,1.0,0.0,0.0,1.0,0.0
1279,0.0,3uoQULcUWfnt6nc6J7Vgai,0.000000,0.836529,0.627,0.090909,0.893350,1.0,0.529989,0.006657,...,0.553562,0.720,0.517270,0.8,0.0,1.0,0.0,0.0,1.0,0.0
1298,0.0,2nbClS09zsIAqNkshg6jnp,0.000000,0.715439,0.745,0.363636,0.853376,0.0,0.526284,0.022590,...,0.626441,0.576,0.543033,0.8,0.0,1.0,0.0,0.0,1.0,0.0


In [28]:
def generate_playlists_vector(complete_data, unique_playlists):
  i=0
  pid_array = []
  summarized_playlists_vector = np.array([])
  for playlist in unique_playlists:
    playlist_data = complete_data[complete_data['pid'] == playlist]
    playlist_data = playlist_data.drop(columns=['pid','track_uri']).sum(axis=0)
    playlist_data = playlist_data.values
    playlist_data = playlist_data.reshape((1,playlist_data.shape[0]))
    if i == 0:
      summarized_playlists_vector = playlist_data
    else: 
      summarized_playlists_vector = np.vstack((summarized_playlists_vector,playlist_data))
    i+=1
    pid_array.append(playlist)
  return summarized_playlists_vector,pid_array


In [29]:
total_unique_playlists = data['pid'].unique()
total_unique_playlists = np.delete(total_unique_playlists,-1)

summarized_playlists,pid_array = generate_playlists_vector(data,total_unique_playlists)

In [30]:
pid_array = np.array(pid_array).reshape((len(pid_array),1))

In [31]:
summarized_playlists = np.hstack((pid_array,summarized_playlists))

In [32]:
summarized_playlists_df = pd.DataFrame(summarized_playlists,
                                       columns=['pid', 'popularity', 'danceability', 'energy', 'key',
                                                'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
                                                'liveness', 'valence', 'tempo', 'time_signature', 'subjectivity_high',
                                                'subjectivity_low', 'subjectivity_medium', 'polarity_Negative',
                                                'polarity_Neutral', 'polarity_Positive'])

In [33]:
summarized_playlists_df.head()

,pid,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,subjectivity_high,subjectivity_low,subjectivity_medium,polarity_Negative,polarity_Neutral,polarity_Positive
0,0.0,24.709677,34.845610,40.6160,23.818182,45.656199,36.0,28.394205,4.368528,0.035103,32.511548,33.4230,27.146630,41.6,13.0,39.0,0.0,3.0,37.0,12.0
1,1.0,17.279570,19.377296,27.1410,15.818182,32.243396,21.0,19.782934,6.352275,8.733356,24.314745,18.5900,21.003525,29.4,3.0,36.0,0.0,4.0,35.0,0.0
2,2.0,14.817204,43.338042,44.3490,29.090909,56.208170,33.0,34.236242,17.291576,0.040861,39.413861,36.1650,31.601788,51.2,18.0,46.0,0.0,6.0,43.0,15.0
3,3.0,27.451613,65.406660,78.2336,58.454545,101.138148,90.0,61.610892,34.601215,25.623249,79.631643,56.8585,68.161845,99.6,29.0,96.0,1.0,12.0,95.0,19.0
4,4.0,5.537634,9.884965,11.0571,5.181818,14.183045,14.0,7.668224,3.024312,1.391290,10.174199,8.3350,9.355963,13.4,8.0,9.0,0.0,3.0,8.0,6.0


In [34]:
from sklearn.metrics.pairwise import cosine_similarity

unique_tracks = data['track_uri'].unique()
def get_song_recommendations(orig_data,playlists_vector,playlist_id,num_of_recommendations):
  playlists_tracks = orig_data[orig_data['pid'] == playlist_id]['track_uri'].unique()
  tracks_not_in_playlist = set(unique_tracks) - set(playlists_tracks)
  non_playlist_df = orig_data[orig_data['track_uri'].isin(tracks_not_in_playlist)]
  non_playlist_df.drop_duplicates(subset=['track_uri'],inplace=True)
  non_playlist_df['sim'] = cosine_similarity(non_playlist_df.drop(columns=['pid','track_uri']).values, playlists_vector[playlists_vector['pid']==playlist_id].drop(columns=['pid']).values)[:,0]
  non_playlist_df = non_playlist_df.sort_values('sim',ascending = False).head(num_of_recommendations)
  return non_playlist_df['track_uri']

In [41]:
def song_names_to_append_to_playlist(data,summarized_playlists_df,playlist_id,num_of_recommendations):
  recommended_track_uri = get_song_recommendations(data,summarized_playlists_df,playlist_id,num_of_recommendations)
  track_names = orignal_data[orignal_data['track_uri'].isin(recommended_track_uri.values)]['track_name'].unique()
  artist_names = orignal_data[orignal_data['track_uri'].isin(recommended_track_uri.values)]['artist_name'].unique()
  album_names = orignal_data[orignal_data['track_uri'].isin(recommended_track_uri.values)]['album_name'].unique()
  # display(track_names)
  print_heading("Top "+str(num_of_recommendations)+" recommendations for Playlist ID: "+str(playlist_id))
  dash_line()
  for song,artist,album in zip(track_names,artist_names,album_names):
    print('"'+song+ '",   by   "' +artist+ '",    Album Name: "' +album+'"')

# Recommendations for Playlist 0     
Top 10

In [42]:
song_names_to_append_to_playlist(data,summarized_playlists_df,0,10)

Top 10 recommendations for Playlist ID: 0
-----------------------------------------------------------------
"Feds Watching",   by   "2 Chainz",    Album Name: "B.O.A.T.S. II #METIME"
"Trap Queen",   by   "Fetty Wap",    Album Name: "Fetty Wap"
"All For One",   by   "High School Musical Cast",    Album Name: "High School Musical 2"
"Heart Of The City (Ain't No Love)",   by   "JAY Z",    Album Name: "The Blueprint"
"Take It Back",   by   "Logic",    Album Name: "Everybody"
"Turn Up The Love",   by   "Far East Movement",    Album Name: "Dirty Bass"
"Go DJ",   by   "Lil Wayne",    Album Name: "Tha Carter"
"What's the Deal",   by   "Young Dolph",    Album Name: "Thinking Out Loud"
"Yahhh!",   by   "Soulja Boy",    Album Name: "souljaboytellem.com"
"Girl Like That",   by   "Every Avenue",    Album Name: "Picture Perfect"


# Recommendations for Playlist 1     
Top 10

In [43]:
song_names_to_append_to_playlist(data,summarized_playlists_df,1,10)

Top 10 recommendations for Playlist ID: 1
-----------------------------------------------------------------
"Plastic Flowers",   by   "The Front Bottoms",    Album Name: "Back On Top"
"Mourning Sound",   by   "Grizzly Bear",    Album Name: "Painted Ruins"
"Focus",   by   "Life of Dillon",    Album Name: "Focus"
"The Past Is A Grotesque Animal",   by   "of Montreal",    Album Name: "Hissing Fauna, Are You The Destroyer?"
"24's",   by   "T.I.",    Album Name: "Trap Muzik"
"Try 'n Hold Me Back",   by   "Rob Bailey & The Hustle Standard",    Album Name: "Battle Tested"
"Intoxicated",   by   "The Cab",    Album Name: "Symphony Soldier"
"Tennessee Mountain Top - Single Version",   by   "Kid Rock",    Album Name: "Tennessee Mountain Top"
"Merry Go Round",   by   "Machine Gun Kelly",    Album Name: "General Admission"
"The Charade",   by   "D'Angelo",    Album Name: "Black Messiah"


# Recommendations for Playlist 10     
Top 10

In [39]:
song_names_to_append_to_playlist(data,summarized_playlists_df,10,10)

Top 10 recommendations for Playlist ID: 10
-----------------------------------------------------------------
CRZY by Kehlani from album: SweetSexySavage
Work by Iggy Azalea from album: The New Classic
Get Throwed by Bun B from album: Trill
Stick Talk by Future from album: DS2
Take It Back by Logic from album: Everybody
24's by T.I. from album: Trap Muzik
Go DJ by Lil Wayne from album: Tha Carter
Gravel Pit by Wu-Tang Clan from album: Gravel Pit
Tell Me You Love Me by Galantis from album: The Aviary
Melbourne Bounce - Deorro Remix by Orkestrated from album: Melbourne Bounce


# Generate Recommendations for Playlist of Your Choice

In [44]:
print("Enter Playlist ID: ")
id = input()
print("Enter number of song recommendations: ")
num = input()

song_names_to_append_to_playlist(data,summarized_playlists_df,int(id),int(num))

Enter Playlist ID: 
34
Enter number of song recommendations: 
12
Top 12 recommendations for Playlist ID: 34
-----------------------------------------------------------------
"Work",   by   "Iggy Azalea",    Album Name: "The New Classic"
"Get Throwed",   by   "Bun B",    Album Name: "Trill"
"Focus",   by   "Life of Dillon",    Album Name: "Focus"
"Decemba (Remix)",   by   "Divine Council",    Album Name: "Council World"
"Gravel Pit",   by   "Wu-Tang Clan",    Album Name: "Gravel Pit"
"China Town",   by   "Migos",    Album Name: "Young Rich N*ggas"
"Sexy And I Know It (Glee Cast Version featuring Ricky Martin)",   by   "Glee Cast",    Album Name: "Glee: The Music, The Complete Season Three"
"24's",   by   "T.I.",    Album Name: "Trap Muzik"
"Get Low - Remastered",   by   "50 Cent",    Album Name: "Get Low"
"En terrasse",   by   "Lefa",    Album Name: "Monsieur Fall"
"Pneumonia",   by   "Danny Brown",    Album Name: "Atrocity Exhibition"
"Lamborghini Angels",   by   "Lupe Fiasco",    Albu